In [2]:
#Criando o mapa e colorindo
#Documentação Plotly: https://plotly.com/python/choropleth-maps/
import pandas as pd
import requests
import json
import folium
from folium.plugins import MarkerCluster
#Planilha1 = pd.read_excel('Dados-Ok-1.xlsx')
Planilha1 = pd.read_excel('Dados.xlsx')
#tabela = Planilha1.set_index('Grupo')
tabela = Planilha1.set_index('ZONA')
#tabela = Planilha1.set_index('REG')
grupos = list(set(tabela.index.values))
#print (tabela)
#print (grupos)

# CODIGO ORIGINAL; TIREI PARA POR CODIGO LOGO ABAIXO
#locais={}
#txt_popup={}

#for grp in grupos:
    #locais['SER_{}'.format(grp)]=Planilha1[Planilha1.ZONA==grp][['LAT','LON']].values.tolist()
    #txt_popup['SER_{}'.format(grp)]=Planilha1[Planilha1.ZONA==grp]['TEXTO'].tolist()

# CODIGO PARA MUDAR TITULO DO MARCADOR
# === PREPARA OS DADOS ANTES DO LOOP ===
locais = {}
txt_popup = {}
nomes_tooltips = {}  # ← nova variável!

for grp in grupos:
    locais[f'SER_{grp}'] = Planilha1[Planilha1.ZONA==grp][['LAT','LON']].values.tolist()
    txt_popup[f'SER_{grp}'] = Planilha1[Planilha1.ZONA==grp]['TEXTO'].tolist()
    nomes_tooltips[f'SER_{grp}'] = Planilha1[Planilha1.ZONA==grp]['Bairro'].tolist()  # ← nova coluna!

#linha original
#mapa = folium.Map(location=[-3.77777,-38.5434], zoom_start=5)
mapa = folium.Map(
    location=[-3.77777,-38.5434],
    tiles='cartodbpositron', #Outras opções: 'Stamen Terrain' (vegetação) e 'Stamen Toner' (rio, lagos e mares)
    zoom_start=12,
)

#for estado in locais.keys():
    ##print(estado)
    #MarkerCluster(locais['{}'.format(estado)],
    #popups = txt_popup['{}'.format(estado)],
    #name = estado).add_to(mapa)

#NOVO CÓDIGO Q SUBSTITUI A PLOTAGEM ACIMA
# === ADICIONA MARCADORES COM POPUP ESTILIZADO ===
for estado in locais.keys():
    cluster = MarkerCluster().add_to(mapa)  # Cria um cluster vazio pra agrupar visualmente

    # Pega os dados do grupo
    coords = locais[estado]        # lista de [lat, lon]
    textos = txt_popup[estado]     # lista de textos (mesmo índice)
    nomes = nomes_tooltips[estado]  # ← pega os nomes

    for (lat, lon), texto in zip(coords, textos, nomes):
        # === MONTA O POPUP COM HTML + CSS ===
        linhas = texto.strip().split('\n')  # separa por quebra de linha no texto original
        html_popup = '<div style="font-family: Arial, sans-serif; font-size: 14px; min-width: 180px; line-height: 1.5;">'
        for linha in linhas:
            if linha.strip():
                # Se quiser negrito no rótulo (ex: "Nome: Fulano")
                if ':' in linha:
                    rotulo, valor = linha.split(':', 1)
                    html_popup += f'<b>{rotulo.strip()}:</b> {valor.strip()}<br>'
                else:
                    html_popup += f'{linha.strip()}<br>'
        html_popup += '</div>'

        # Cria o popup
        popup = folium.Popup(html_popup, max_width=300)

        # Cria o marcador
        folium.Marker(
            location=[lat, lon],
            #popup=popup, # LINHA ORIGINAL
            popup=folium.Popup(html, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign'),
            #tooltip=estado  # aparece ao passar o mouse  # LINHA ORIGINAL
            tooltip=nome.strip()  # ← agora é o nome do local!
        ).add_to(cluster)


# CARREGA GEOJSON DOS BAIRROS
# atencao: tive q colocar o ENCODING="LATIN-1" pq deu erro: 'charmap' codec can't decode byte 0x81
with open('Fortal-Bai.geojson', encoding="latin-1") as file:
    state_geo = json.load (file)

type(state_geo) # O arquivo GeoJSON é transformado em um dicionário após a aplicação do método .json()
#print (state_geo)

#parametros do GEOJSON
state_data = pd.read_excel("Dados.xlsx")
## state_data = pd.read_csv("RegionalCSV.csv")
state_data

#colorindo o mapa - LINHA FUNCIONANDO
#folium.Choropleth(
    #geo_data=state_geo,
    #data=state_data,
    ##columns=["Codigos", "Variavel"],
    #columns=["Bairros", "Votos"],
    #key_on="feature.properties.id",
    #fill_color="BuGn",
    #fill_opacity=0.7,
    #line_opacity=0.5,
    #line_color="black",
    #legend_name="Total: 4344 Votos",
#).add_to(mapa)

#colorindo o mapa; ELE JA SOMA A COLUNA "Votos"
folium.Choropleth(
    geo_data=state_geo,
    data=state_data,
    #columns=["Bairros", "Votos"],
    columns=["Bairros", "Votos"],
    nan_fill_color="red",
    nan_fill_opacity=0.4,
    key_on="feature.properties.id",
    fill_color="BuGn",
    fill_opacity=0.7,
    line_opacity=0.5,
    line_color="black",
    legend_name="Total: 4.344 Votos",
).add_to(mapa)

##folium.LayerControl().add_to(mapa)
mapa.save('map-aly-ED-2.html')
mapa


KeyError: 'NOME_LOCAL'